<a href="https://colab.research.google.com/github/Karthick47v2/mock-buddy/blob/main/face_landmark_model_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn.model_selection
import tensorflow
from tensorflow import keras

In [2]:
y_train = np.load('/content/gdrive/MyDrive/data-v/y_train30_AUG.npy')
frames = np.load('/content/gdrive/MyDrive/data-v/frames30_AUG.npy')

In [3]:
img_size = 224

In [5]:
# creating sequential model
model = keras.models.Sequential()

# using MobileNetV2 as feature extractor // include_top=False -> discarding last few layers and going to add custom layers for landmark detection
mn_v2 = keras.applications.MobileNetV2(input_shape=(224,224,3), include_top=False, weights='imagenet')

In [6]:
model.add(mn_v2)
model.add(keras.layers.GlobalAveragePooling2D())
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.Dense(136))

model.build(input_shape=(None, 224, 224, 3))

In [11]:
###########################
mobilenetv2 = keras.applications.MobileNetV2(input_shape=(img_size,img_size,3), include_top=False, weights='imagenet')
mobilenetv2.trainable = False

input = keras.layers.Input((img_size,img_size,3))

x = keras.applications.mobilenet_v2.preprocess_input(input)
x = mobilenetv2(x)
# x = keras.layers.Dropout(0.5)(x)
x = keras.layers.SeparableConv2D(136, kernel_size=5, strides=1, activation="relu")(x)
output = keras.layers.SeparableConv2D(136, kernel_size=3, strides=1, activation="sigmoid")(x)

model = keras.Model(input, output)
#####################


In [12]:
model.compile(optimizer='adam', loss="mean_squared_error", metrics=["mae", "accuracy"])

chk_path = "trained_model/model"
model_checkpoint = keras.callbacks.ModelCheckpoint(filepath=chk_path,
                                                   monitor="val_mae",
                                                   mode="auto",
                                                   save_best_only=True,
                                                   save_weights_only=True
                                                   )

In [13]:
reduce_lr = keras.callbacks.ReduceLROnPlateau(factor=0.9,
                                              monitor="val_mae",
                                              mode="auto",
                                              cooldown=0,
                                              patience=5,
                                              verbose=1,
                                              min_lr=1e-6
                                              )

In [5]:
train_frames, val_frames, train_lm, val_lm = sklearn.model_selection.train_test_split(frames, y_train, test_size=0.2, random_state=42)

In [6]:
frames = []
y_train = []

In [14]:
EPOCHS=300
BATCH_SIZE=32

In [ ]:
history=model.fit(train_frames,
                  train_lm,
                  validation_data=(val_frames, val_lm),
                  batch_size=BATCH_SIZE,
                  epochs=EPOCHS,
                  callbacks=[model_checkpoint, reduce_lr]
                  )

In [ ]:
model.save('/content/gdrive/MyDrive/data-v/mymodel')

In [12]:
model = keras.models.load_model('/content/gdrive/MyDrive/data-v/mymodel')

In [13]:
test_frames = np.load('/content/gdrive/MyDrive/data-v/test_frames.npy')

In [ ]:
fig, arr = plt.subplots(6,5,figsize=(20,20))

for i in range(1, 30):
  r = i // 5
  c = i % 5

  sam = test_frames[i]
  pred = model.predict( test_frames[ i : i +1  ] ) 
  x=np.reshape(pred[:,:,:,np.arange(0,136,2)],(68))*img_size
  y=np.reshape(pred[:,:,:,np.arange(1,136,2)],(68))*img_size
  arr[r][c].imshow(sam)
  arr[r][c].scatter( x,y, c='yellow',s=6)